In [ ]:
import scipp as sc
import numpy as np
import matplotlib.pyplot as plt
import plopp as pp
%matplotlib widget

#TODO Make a general data loader
## Load vanadium data
number_of_Q_points=16

#Preallocate
intensity_values=np.zeros((number_of_Q_points,1024))
error_values=np.zeros((number_of_Q_points,1024))

# Load data into a matrix
for Q in range(16):
    filename = '../../IN16b_GGG_data/vanadium_Q' +str(Q+1) +'.dat'

    data_array = np.loadtxt(filename)
    energy_values=data_array[:, 0] #should be the same for all Q
    # EnergyValues[Q,:]=data_array[:, 0]
    intensity_values[Q,:]=data_array[:,1]
    error_values[Q,:]=data_array[:,2]

# Define energy, q and intensity as scipp variables with units, and make a DataArraw
energy=sc.array(dims=['energy'],values=energy_values/1000,unit='meV')
Q=sc.array(dims=['Q'],values=range(number_of_Q_points))
intensity=sc.array(dims=['Q','energy'],values=intensity_values,variances=error_values*error_values) #The variance is the square of the uncertainty!

vanadium_data = sc.DataArray(data=intensity, coords={'Q':Q,'energy': energy})

## Bin and plot data 

Q_bins=16
energy_bins=sc.scalar(1e-3*0.2, unit='meV')

intensity_min=0.0
intensity_max=0.06

energy_min = -0.02 * sc.Unit('meV')
energy_max = 0.02 * sc.Unit('meV')

# da.coords['Energy'] = da.coords['Energy'].to(unit='micro*eV') #optional change the scale to mueV
binned_vanadium_data=vanadium_data.flatten(to='dummy').bin(energy=energy_bins,Q=Q_bins).bins.mean() #can add .plot() to plot it

binned_vanadium_data.plot()

binned_vanadium_data

In [ ]:

# Slice the binned data
pp.slicer(binned_vanadium_data['energy',energy_min:energy_max],vmin=intensity_min,vmax=intensity_max,
     keep=['energy'],
     linestyle='none',
     marker='o',
     markerfacecolor='none',
     color='k'
)





In [ ]:
# Fit all data to the same function
# TODO: Make a library of functions instead of defining them inline.
# TODO: Add a Lorentzian tail to the resolution function
def resolution_function_scipp(energy, energy_offset,res_gauss1_area, res_gauss1_sigma,res_BG):
    x=energy
    x=x-energy_offset

    y=res_gauss1_area*1/np.sqrt(2*np.pi)/res_gauss1_sigma*sc.exp(-0.5*(x/res_gauss1_sigma)**2  )+res_BG
    return y


from scipp import curve_fit
# from scipp.scipy.optimize import curve_fit
# TODO Add the option of providing start guess for each Q point, e.g. by using the value from higher or lower Q
popt, _ = curve_fit(['energy'], resolution_function_scipp, vanadium_data, 
                    p0 = {'res_gauss1_area' : 5e-5*sc.Unit('meV'), 
                          'res_gauss1_sigma': 4e-4 * sc.Unit('meV'),
                          'energy_offset'   : 0 * sc.Unit('meV'),
                          'res_BG'          :1e-5})

#Calculate the fit to display it
#TODO Make it possible to calculate the fit on a more dense set of points than the data
vanadium_fit=resolution_function_scipp(sc.values(vanadium_data.coords['energy']),
                                      energy_offset     =sc.values(popt['energy_offset']),
                                      res_gauss1_area   =sc.values(popt['res_gauss1_area']),
                                      res_gauss1_sigma  =sc.values(popt['res_gauss1_sigma']),
                                      res_BG            =sc.values(popt['res_BG']))
vanadium_fit.coords['energy']=energy


data_and_fit=sc.DataGroup({'Data': binned_vanadium_data,
                            'Fit': vanadium_fit})

pp.slicer(data_and_fit['energy',energy_min:energy_max],vmin=intensity_min,vmax=intensity_max,
     keep=['energy'],
     linestyle={'Data': 'none',   'Fit': '-'},
     marker={'Data': 'o', 'Fit':'none'},
     markerfacecolor={'Data': 'none', 'Fit' :'red'},
    color={'Data': 'black',  'Fit': 'red'}
)


